In [9]:
import os
import random
import math
from PIL import Image
import numpy as np

In [10]:
def extract_binary_features(image_path, size=(16, 16), threshold=128):
    image = Image.open(image_path).convert("L").resize(size)
    data = np.array(image)
    binary = (data > threshold).astype(int)
    return binary.flatten().tolist()

In [11]:
def load_dataset_binary(dataset_path):
    data = []
    for label in os.listdir(dataset_path):
        label_path = os.path.join(dataset_path, label)
        if not os.path.isdir(label_path): continue
        for filename in os.listdir(label_path):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                path = os.path.join(label_path, filename)
                features = extract_binary_features(path)
                data.append({'label': label, 'features': features, 'filename': filename})
    return data

In [12]:
def train_naive_bayes_binary(data):
    model = {}
    total_per_class = {}
    for sample in data:
        label = sample['label']
        if label not in model:
            model[label] = {'counts': [ [0, 0] for _ in range(len(sample['features'])) ]}
            total_per_class[label] = 0
        total_per_class[label] += 1
        for i, val in enumerate(sample['features']):
            model[label]['counts'][i][val] += 1  # val = 0 or 1
    model['total'] = sum(total_per_class.values())
    model['per_class'] = total_per_class
    return model

In [13]:
def predict_binary(model, features):
    max_prob = -math.inf
    best_label = None

    for label, data in model.items():
        if label == 'total' or label == 'per_class': continue
        log_prob = math.log(model['per_class'][label] / model['total'])  # prior
        for i, val in enumerate(features):
            count_0, count_1 = data['counts'][i]
            total = count_0 + count_1
            if total == 0:
                prob = 0.5  # default
            else:
                prob = (count_1 if val == 1 else count_0) / total
            log_prob += math.log(prob + 1e-6)  # biar ga log(0)
        if log_prob > max_prob:
            max_prob = log_prob
            best_label = label
    return best_label


In [14]:
def evaluate_binary_model(model, test_data):
    benar = 0
    for i, sample in enumerate(test_data):
        pred = predict_binary(model, sample['features'])
        actual = sample['label']
        if pred == actual:
            benar += 1
        print(f"[{i+1}] File: {sample['filename']:20s} | Asli: {actual:10s} → Prediksi: {pred:10s}")
    akurasi = benar / len(test_data) * 100
    print(f"\n🎯 Akurasi: {akurasi:.2f}%")

In [15]:
if __name__ == "__main__":
    dataset_path = "dataset"
    data = load_dataset_binary(dataset_path)
    random.shuffle(data)

    split = int(0.8 * len(data))
    train_data = data[:split]
    test_data = data[split:]

    model = train_naive_bayes_binary(train_data)
    evaluate_binary_model(model, test_data)

[1] File: CC_HE_1221887081_SituationalDepression-650x428.jpg | Asli: sedih      → Prediksi: sedih     
[2] File: angry-people-screaming-260nw-292800785.jpg | Asli: marah      → Prediksi: senang    
[3] File: Angry-Person-PNG-Free-Download.png | Asli: marah      → Prediksi: marah     
[4] File: 640px-Crying-girl.jpg | Asli: sedih      → Prediksi: sedih     
[5] File: png-transparent-boxing-man-people-an-angry-person-the-man.png | Asli: marah      → Prediksi: marah     
[6] File: 81Vk4C.jpg           | Asli: senang     → Prediksi: senang    
[7] File: 35438_hd.jpg         | Asli: senang     → Prediksi: senang    
[8] File: Anger_Controlls_Him.jpg | Asli: marah      → Prediksi: sedih     
[9] File: happy-people2.jpg    | Asli: senang     → Prediksi: marah     
[10] File: man-portrait-contemplative-sad-looking-at-camera-732x549-thumbnail.jpg | Asli: sedih      → Prediksi: sedih     
[11] File: happy-people12.jpg   | Asli: senang     → Prediksi: marah     
[12] File: Depositphotos_6311024_X